In [1]:
import sys
import math
import csv
import random
import json 
import time

try:
    jparams = json.load(open('params.json'))
except:
    print("ERROR: something is wrong with the params.json file.")
    sys.exit()
    
#-- store the input 3D points in list
list_pts_3d = []
with open(jparams['input-file']) as csvfile:
    r = csv.reader(csvfile, delimiter=' ')
    header = next(r)
    for line in r:
        p = list(map(float, line)) #-- convert each str to a float
        assert(len(p) == 3)
        list_pts_3d.append(p)

In [2]:
import math
import numpy
import scipy.spatial

In [77]:
class BoundingBox:
    """A 2D bounding box"""
    
    def __init__(self, points):
        if len(points) == 0:
            raise ValueError("Can't compute bounding box of empty list")
        
        self.minx, self.miny = float("inf"), float("inf")
        self.maxx, self.maxy = float("-inf"), float("-inf")
        for x, y, *_ in points:
            # Set min coords
            if x < self.minx:
                self.minx = x
            if y < self.miny:
                self.miny = y
            # Set max coords
            if x > self.maxx:
                self.maxx = x
            elif y > self.maxy:
                self.maxy = y
                
    @property
    def width(self):
        return self.maxx - self.minx
    @property
    def height(self):
        return self.maxy - self.miny
    def __repr__(self):
        return "BoundingBox(X: {} - {}, Y: {} - {})".format(
            self.minx, self.maxx, self.miny, self.maxy)
      
bbox = BoundingBox(list_pts_3d)
bbox

BoundingBox(X: 2.0 - 252.0, Y: 3.0 - 253.0)

In [78]:
class Raster:
  """Simple raster based on ESRI ASCII schema"""
  
  def __init__(self, bbox, cell_size, no_data=-9999):
    self.ncols = int(bbox.width // cell_size + 1)
    self.nrows = int(bbox.height // cell_size + 1)
    self.xllcenter = bbox.minx
    self.yllcenter = bbox.miny
    self.cell_size = cell_size
    self.no_data = no_data
    
    # initialize list of values with no_data
    self.values = [self.no_data] * self.ncols * self.nrows
  
  @property
  def centers(self):
    """Cell center coordinates"""
    xulcenter = self.xllcenter + self.cell_size * self.nrows
    for i in range(self.ncols):
      for j in range(self.nrows):
        x = xulcenter - i * self.cell_size
        y = self.yllcenter + j * self.cell_size
        yield (x,y)
    
  def to_ascii(self):
    rows = [
      "NCOLS %d" % self.ncols,
      "NROWS %d" % self.nrows,
      "XLLCENTER %s" % self.xllcenter,
      "YLLCENTER %s" % self.yllcenter,
      "CELLSIZE %s" % self.cell_size,
      "NODATA_VALUE %s" % self.no_data,
      ' '.join( (str(x) for x in self.values) )
    ]
    return '\n'.join(rows)

In [79]:
bbox = BoundingBox(list_pts_3d)
raster = Raster(bbox, 30)
type(raster.to_ascii())

str

In [80]:
list_pts_2d = [(x,y) for x,y,z in list_pts_3d]
list_pts_z = [(z) for x,y,z in list_pts_3d]
kdtree = scipy.spatial.KDTree(list_pts_2d)

raster.values = []
for center in raster.centers:
    samples = kdtree.query_ball_point(center, 10)
    coords = [list_pts_2d[i] for i in samples]
    values = [list_pts_z[i] for i in samples]

    # calculate distances (without sqrt) and solve 0 cases
    dists = [(center[0]-c[0])**2 + (center[1]-c[1])**2 for c in coords]
    if 0 in dists:
      index = dists.index(0)
      raster.values.append(list_pts_z[index])
    # if no zero distance cases, weight the values
    else:  
      weights = [1/math.sqrt(d) for d in dists]
      weights_norm = [w/sum(weights) for w in weights]
      result = sum([norm*z for norm, z in zip(weights_norm, values)])
      raster.values.append(result)

    
raster.values

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 109.59004820692088,
 102.75899845740938,
 116.28725,
 51.905826630209745,
 44.13321117109679,
 43.50950732657751,
 41.78370432697501,
 116.28725,
 119.77059075207535,
 126.70687110156923,
 122.76253682545187,
 96.40375363470822,
 67.35789694339519,
 66.08045308821609,
 61.16968730109092,
 43.53856449341872,
 37.956936741572704,
 116.18125740365231,
 128.86368828198573,
 129.85945123729195,
 113.18102918054383,
 101.23352009150477,
 90.17161137379979,
 64.33155484801158,
 49.91590015993549,
 49.48806126936641,
 97.57074955474008,
 117.68621317881144,
 138.24763310084532,
 137.48237672908957,
 123.32890013583912,
 103.01627984032949,
 88.08659740900694,
 86.10820012434652,
 74.22506159815356,
 87.46374218436816,
 113.20525892758661,
 132.54260641753535,
 148.59387950007533,
 145.38581712542808,
 116.18964765107313,
 110.83776073252854,
 104.69675563661553,
 91.51295613101244,
 83.94869730808995,
 102.4993460071837,
 123.18208830675093,
 136.735493

In [82]:
a = [1,3,4,0]
a.index(0)

3